# PREPARATION

In [ ]:
import pandas as pd

df = pd.read_csv('train.csv', index_col='PassengerId')
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [ ]:
df.isnull().any()

Survived    False
Pclass      False
Name        False
Sex         False
Age          True
SibSp       False
Parch       False
Ticket      False
Fare        False
Cabin        True
Embarked     True
dtype: bool

In [ ]:
df.pop('Name')
df.pop('Ticket')
df.pop('Cabin')

PassengerId
1       NaN
2       C85
3       NaN
4      C123
5       NaN
       ... 
887     NaN
888     B42
889     NaN
890    C148
891     NaN
Name: Cabin, Length: 891, dtype: object

In [ ]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,male,22.0,1,0,7.2500,S
2,1,1,female,38.0,1,0,71.2833,C
3,1,3,female,26.0,0,0,7.9250,S
4,1,1,female,35.0,1,0,53.1000,S
5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
887,0,2,male,27.0,0,0,13.0000,S
888,1,1,female,19.0,0,0,30.0000,S
889,0,3,female,NaN,1,2,23.4500,S


In [ ]:
cat_mask = (df.dtypes==object)
cat_cols = df.columns[cat_mask].tolist()
df_cat = df[cat_cols]
df_num = df.drop(cat_cols, axis=1)

In [ ]:
df_cat

,Sex,Embarked
PassengerId,,
1,male,S
2,female,C
3,female,S
4,female,S
5,male,S
...,...,...
887,male,S
888,female,S
889,female,S


In [ ]:
from sklearn.impute import SimpleImputer
imp_cat = SimpleImputer(strategy='most_frequent')
df_cat = pd.DataFrame(imp_cat.fit_transform(df_cat),
                      columns=df_cat.columns, index=df_cat.index)

In [ ]:
df_cat.isnull().any()

Sex         False
Embarked    False
dtype: bool

In [ ]:
from sklearn import preprocessing
from collections import defaultdict

d = defaultdict(preprocessing.LabelEncoder)

df_cat_le = df_cat.apply(lambda col: d[col.name].fit_transform(col))
df_cat_le

,Sex,Embarked
PassengerId,,
1,1,2
2,0,0
3,0,2
4,0,2
5,1,2
...,...,...
887,1,2
888,0,2
889,0,2


In [ ]:
ohe = preprocessing.OneHotEncoder(sparse=False)
df_cat_ohe = pd.DataFrame(ohe.fit_transform(df_cat),
                          columns=ohe.get_feature_names(df_cat.columns.tolist()),
                          index=df_cat.index)

In [ ]:
df_cat_ohe.pop('Sex_female')

PassengerId
1      0.0
2      1.0
3      1.0
4      1.0
5      0.0
      ... 
887    0.0
888    1.0
889    1.0
890    0.0
891    0.0
Name: Sex_female, Length: 891, dtype: float64

In [ ]:
df_cat_ohe

,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,
1,1.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0
5,1.0,0.0,0.0,1.0
...,...,...,...,...
887,1.0,0.0,0.0,1.0
888,0.0,0.0,0.0,1.0
889,0.0,0.0,0.0,1.0


In [ ]:
imp_num = SimpleImputer(strategy='mean')
df_num = pd.DataFrame(imp_num.fit_transform(df_num),
                      columns=df_num.columns, index=df_num.index)
df_num

,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,,,,,,
1,0.0,3.0,22.000000,1.0,0.0,7.2500
2,1.0,1.0,38.000000,1.0,0.0,71.2833
3,1.0,3.0,26.000000,0.0,0.0,7.9250
4,1.0,1.0,35.000000,1.0,0.0,53.1000
5,0.0,3.0,35.000000,0.0,0.0,8.0500
...,...,...,...,...,...,...
887,0.0,2.0,27.000000,0.0,0.0,13.0000
888,1.0,1.0,19.000000,0.0,0.0,30.0000
889,0.0,3.0,29.699118,1.0,2.0,23.4500


In [ ]:
df_num.isnull().any()

Survived    False
Pclass      False
Age         False
SibSp       False
Parch       False
Fare        False
dtype: bool

In [ ]:
df_preprocessed = pd.merge(left=df_cat_ohe,
                           right=df_num,
                           on='PassengerId')
class_col = df_preprocessed.pop('Survived')
df_preprocessed

,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass,Age,SibSp,Parch,Fare
PassengerId,,,,,,,,,
1,1.0,0.0,0.0,1.0,3.0,22.000000,1.0,0.0,7.2500
2,0.0,1.0,0.0,0.0,1.0,38.000000,1.0,0.0,71.2833
3,0.0,0.0,0.0,1.0,3.0,26.000000,0.0,0.0,7.9250
4,0.0,0.0,0.0,1.0,1.0,35.000000,1.0,0.0,53.1000
5,1.0,0.0,0.0,1.0,3.0,35.000000,0.0,0.0,8.0500
...,...,...,...,...,...,...,...,...,...
887,1.0,0.0,0.0,1.0,2.0,27.000000,0.0,0.0,13.0000
888,0.0,0.0,0.0,1.0,1.0,19.000000,0.0,0.0,30.0000
889,0.0,0.0,0.0,1.0,3.0,29.699118,1.0,2.0,23.4500


## Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

fs_k_best_chi2 = SelectKBest(chi2, k=4)
fs_k_best_chi2.fit(df_preprocessed, class_col)
col_filter = fs_k_best_chi2.get_support()
df_k_best_chi2 = df_preprocessed.iloc[:, col_filter]
df_k_best_chi2

,Sex_male,Pclass,Age,Fare
PassengerId,,,,
1,1.0,3.0,22.000000,7.2500
2,0.0,1.0,38.000000,71.2833
3,0.0,3.0,26.000000,7.9250
4,0.0,1.0,35.000000,53.1000
5,1.0,3.0,35.000000,8.0500
...,...,...,...,...
887,1.0,2.0,27.000000,13.0000
888,0.0,1.0,19.000000,30.0000
889,0.0,3.0,29.699118,23.4500


In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

fs_k_best_mi = SelectKBest(mutual_info_classif, k=4)
fs_k_best_mi.fit(df_preprocessed, class_col)
col_filter = fs_k_best_mi.get_support()
df_k_best_mi = df_preprocessed.iloc[:, col_filter]
df_k_best_mi

,Sex_male,Pclass,Age,Fare
PassengerId,,,,
1,1.0,3.0,22.000000,7.2500
2,0.0,1.0,38.000000,71.2833
3,0.0,3.0,26.000000,7.9250
4,0.0,1.0,35.000000,53.1000
5,1.0,3.0,35.000000,8.0500
...,...,...,...,...
887,1.0,2.0,27.000000,13.0000
888,0.0,1.0,19.000000,30.0000
889,0.0,3.0,29.699118,23.4500


In [ ]:
df_preprocessed

,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass,Age,SibSp,Parch,Fare
PassengerId,,,,,,,,,
1,1.0,0.0,0.0,1.0,3.0,22.000000,1.0,0.0,7.2500
2,0.0,1.0,0.0,0.0,1.0,38.000000,1.0,0.0,71.2833
3,0.0,0.0,0.0,1.0,3.0,26.000000,0.0,0.0,7.9250
4,0.0,0.0,0.0,1.0,1.0,35.000000,1.0,0.0,53.1000
5,1.0,0.0,0.0,1.0,3.0,35.000000,0.0,0.0,8.0500
...,...,...,...,...,...,...,...,...,...
887,1.0,0.0,0.0,1.0,2.0,27.000000,0.0,0.0,13.0000
888,0.0,0.0,0.0,1.0,1.0,19.000000,0.0,0.0,30.0000
889,0.0,0.0,0.0,1.0,3.0,29.699118,1.0,2.0,23.4500


# MODELLING

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_preprocessed, class_col,
                                                    test_size=0.3, random_state=1)
X_train

,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass,Age,SibSp,Parch,Fare
PassengerId,,,,,,,,,
115,0.0,1.0,0.0,0.0,3.0,17.000000,0.0,0.0,14.4583
875,0.0,1.0,0.0,0.0,2.0,28.000000,1.0,0.0,24.0000
77,1.0,0.0,0.0,1.0,3.0,29.699118,0.0,0.0,7.8958
877,1.0,0.0,0.0,1.0,3.0,20.000000,0.0,0.0,9.8458
675,1.0,0.0,0.0,1.0,2.0,29.699118,0.0,0.0,0.0000
...,...,...,...,...,...,...,...,...,...
716,1.0,0.0,0.0,1.0,3.0,19.000000,0.0,0.0,7.6500
768,0.0,0.0,1.0,0.0,3.0,30.500000,0.0,0.0,7.7500
73,1.0,0.0,0.0,1.0,2.0,21.000000,0.0,0.0,73.5000


## Decission trees

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth=4, min_samples_split=5,
                              min_samples_leaf=3, random_state=1)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)


from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

conf = pd.DataFrame(confusion_matrix(y_test, y_pred),
                    columns = ['Predicted 0', 'Predicted 1'],
                    index = ['True 0', 'True1'])
conf

              precision    recall  f1-score   support

         0.0       0.74      0.93      0.83       153
         1.0       0.86      0.57      0.69       115

    accuracy                           0.78       268
   macro avg       0.80      0.75      0.76       268
weighted avg       0.79      0.78      0.77       268



,Predicted 0,Predicted 1
True 0,142,11
True1,49,66


## Random Forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)


from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

conf = pd.DataFrame(confusion_matrix(y_test, y_pred),
                    columns = ['Predicted 0', 'Predicted 1'],
                    index = ['True 0', 'True1'])
conf

              precision    recall  f1-score   support

         0.0       0.75      0.88      0.81       153
         1.0       0.79      0.62      0.69       115

    accuracy                           0.76       268
   macro avg       0.77      0.75      0.75       268
weighted avg       0.77      0.76      0.76       268



,Predicted 0,Predicted 1
True 0,134,19
True1,44,71


## SVC

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
df_num_sc = pd.DataFrame(mms.fit_transform(df_num), columns=df_num.columns, index=df_num.index)
df_preprocessed_sc = pd.merge(left=df_cat_ohe, right=df_num_sc, on='PassengerId')

X_train_sc, X_test_sc, y_train_sc, y_test_sc = train_test_split(df_preprocessed_sc, class_col,
                                                                test_size=0.3, random_state=1)

svm = SVC()
svm.fit(X_train_sc, y_train_sc)
y_pred_sc = svm.predict(X_test_sc)



from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(classification_report(y_test_sc, y_pred_sc))

conf = pd.DataFrame(confusion_matrix(y_test_sc, y_pred_sc),
                    columns = ['Predicted 0', 'Predicted 1'],
                    index = ['True 0', 'True1'])
conf

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       153
         1.0       1.00      1.00      1.00       115

    accuracy                           1.00       268
   macro avg       1.00      1.00      1.00       268
weighted avg       1.00      1.00      1.00       268



,Predicted 0,Predicted 1
True 0,153,0
True1,0,115
